In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Lectura pagos Bradesco
pagos_brad = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Bradesco/Pagos/ENRIQUEZ Acumulado de pagos Diciembre de 2020.xlsx')
pagos_brad = pagos_brad.iloc[:,0:9]
pagos_brad = pagos_brad.drop_duplicates()
pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [5]:
#Lectura pagos Liverpool
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
for i in sheet_liv:
    pagos_liv_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Liverpool/Pagos/PAGOS DIC LIVERPOOL .xlsx',sheet_name = str(i))
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
pagos_liv = pagos_liv.drop_duplicates()
pagos_liv = pagos_liv.iloc[:,0:6]
pagos_liv.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
#try: 
pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
#except: pagos_liv['Fecha Pago'] 
#pagos_liv['Fecha Pago'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Pago']]
try: 
    pagos_liv['Fecha Prom'] = pd.to_datetime(pagos_liv['Fecha Prom'],dayfirst=True)
except: pagos_liv['Fecha Prom'] = pagos_liv['Fecha Prom']
pagos_liv['Fecha Prom'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Prom']]
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
pagos_liv2 = pagos_liv2.loc[pagos_liv2['Fecha Pago']>=mes3]

In [6]:
#Lectura pagos Credifiel
sheet_cred = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for i in sheet_cred:
    pagos_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/'+i+'')
    pagos_cred = pd.concat([pagos_cred,pagos_cred_new],ignore_index=True)
pagos_cred = pagos_cred.iloc[:,0:5]
pagos_cred2 = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred2 = pd.DataFrame(pagos_cred2.to_records())

In [7]:
#Lectura asignacion Credifiel
sheet_cred2 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Asignacion/')
asignacion_cred = pd.DataFrame()
for j in sheet_cred2:
    asignacion_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Asignacion/'+j+'')
    asignacion_cred = pd.concat([asignacion_cred,asignacion_cred_new],ignore_index=True)
asignacion_cred = asignacion_cred.loc[:,['CREDITO','ARANCEL']]

In [8]:
#Lectura pagos Banco Azteca
sheet_baz_renta = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta')
pagos_baz_renta = pd.DataFrame()
for i in sheet_baz_renta:
    pagos_baz_renta_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta/'+i+'')
    pagos_baz_renta = pd.concat([pagos_baz_renta,pagos_baz_renta_new],ignore_index=True)
pagos_baz_renta = pagos_baz_renta.drop_duplicates()
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FNMONTOPAGADO']>0]
pagos_baz_renta['FDFECHAABONO'] = pd.to_datetime(pagos_baz_renta['FDFECHAABONO'],dayfirst=True)
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FDFECHAABONO']>=mes3]
pagos_baz_renta2 = pd.pivot_table(pagos_baz_renta,index=['CAMPANAID','FDFECHAABONO','FIIDPERIODO','FNSCOMPROMISO','FCEMPNUMCORTE','FNMONTOPROMETIDO'],values=['FNMONTOPAGADO'],aggfunc=np.sum)
pagos_baz_renta2 = pd.DataFrame(pagos_baz_renta2.to_records())
pagos_baz_renta2['Cliente'] = 'Renta Azteca'

In [9]:
#Leemos facturacion Banco Azteca
sheet_baz2 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/')
fact_baz = pd.DataFrame()
for k in sheet_baz2:
    fact_baz_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/'+k+'',encoding='latin-1')
    fact_baz = pd.concat([fact_baz,fact_baz_new],ignore_index=True)
fact_baz = fact_baz.drop_duplicates()
fact_baz = fact_baz.loc[fact_baz['Número Semana']>=prim_sem_mes]
fact_baz['Final de Semana'] = pd.to_datetime(fact_baz['Final de Semana'],dayfirst=True)
fact_baz['Inicio de Semana'] = pd.to_datetime(fact_baz['Inicio de Semana'],dayfirst=True)
fact_baz['Cliente'] = 'Banco Azteca'

In [10]:
#Lectura pagos Banco Azteca Recovery
sheet_baz_recovery = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/')
pagos_baz_recovery = pd.DataFrame()
for k in sheet_baz_recovery:
    pagos_baz_recovery_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/'+k+'',sep='|')
    pagos_baz_recovery = pd.concat([pagos_baz_recovery,pagos_baz_recovery_new],ignore_index=True)
pagos_baz_recovery['Cartera'] = 'Recovery'
pagos_baz_recovery['DIA'] = pd.to_datetime(pagos_baz_recovery['DIA'],dayfirst=True)
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery['credito'] = pagos_baz_recovery['PAIS'].map(str)+'-'+pagos_baz_recovery['CANAL'].map(str)+'-'+pagos_baz_recovery['SUCURSAL'].map(str)+'-'+pagos_baz_recovery['FOLIO'].map(str)
pagos_baz_recovery = pagos_baz_recovery.drop_duplicates()

In [11]:
#Lectura pagos Banco Azteca 26-39
sheet_baz_26 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/')
pagos_baz_26 = pd.DataFrame()
for j in sheet_baz_26:
    pagos_baz_26_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/'+j+'',sep='|')
    pagos_baz_26 = pd.concat([pagos_baz_26,pagos_baz_26_new],ignore_index=True)
pagos_baz_26['cartera'] = '26-39'
pagos_baz_26['DIA'] = pd.to_datetime(pagos_baz_26['DIA'],dayfirst=True)
pagos_baz_26 = pagos_baz_26.loc[pagos_baz_26['DIA']>=mes3]
pagos_baz_26['credito'] = pagos_baz_26['PAIS'].map(str)+'-'+pagos_baz_26['CANAL'].map(str)+'-'+pagos_baz_26['SUCURSAL'].map(str)+'-'+pagos_baz_26['FOLIO'].map(str)
pagos_baz_26 = pagos_baz_26.drop_duplicates()

In [12]:
#Asignamos valores a los parametros"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [13]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [56]:
meta = pd.DataFrame([('Liverpool',1000000),('Bradesco',1200000),('Credifiel',120000)])
meta.columns = ['Cliente','Meta']

In [14]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 1204704, .07
c04pv2, arancel_4pv2 = 1807056, .09
c04pv3, arancel_4pv3 = 2409407, .12
c05pv1, arancel_5pv1 = 662269, .09
c05pv2, arancel_5pv2 = 993403, .10
c05pv3, arancel_5pv3 = 1324538, .14
c06pv1, arancel_6pv1 = 807172, .10
c06pv2, arancel_6pv2 = 1210759, .11
c06pv3, arancel_6pv3 = 1614345, .16
a1, arancel_a1 = 260190, .27
a2, arancel_a2 = 390285, .30
a3, arancel_a3 = 520381, .33
b1, arancel_b1 = 215083, .25
b2, arancel_b2 = 322624, .28
b3, arancel_b3 = 430165, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 626634, .19
c22, arancel_c22 = 939951, .22
c23, arancel_c23 = 1253267, .25
c31, arancel_c31 = 246425, .21
c32, arancel_c32 = 369637, .24
c33, arancel_c33 = 492849, .27
c41, arancel_c41 = 123529, .23
c42, arancel_c42 = 185294, .26
c43, arancel_c43 = 247059, .29

In [15]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None):
    if from_date is None:
        sql_mes = cursor.callproc(table,)
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date])
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

In [16]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

# Reporte Liverpool

In [17]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv_mes = consulta_sql('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv_mes = gestion_liv_mes.loc[gestion_liv_mes['dictamen']=='PROMESA']
gestion_liv_mes[['credito','telefono']] = transform_numeric(gestion_liv_mes,['credito','telefono'])
gestion_liv_mes['anio'] = [int(x[:4]) for x in gestion_liv_mes['fecha_promesa']]
gestion_liv_mes = gestion_liv_mes.loc[(gestion_liv_mes['anio']>2010)&(gestion_liv_mes['anio']<2100)]
gestion_liv_mes['fecha_promesa'] = pd.to_datetime(gestion_liv_mes['fecha_promesa'],dayfirst=False)

In [18]:
#Hacemos el consolidado
gestion_liv_mes[['credito','monto_promesa']] = transform_numeric(gestion_liv_mes,['credito','monto_promesa'])
gestion_liv_mes = gestion_liv_mes[(gestion_liv_mes['monto_promesa']>50)]
gestion_liv_mes = gestion_liv_mes[(gestion_liv_mes['monto_promesa']<500000)]
gestion_liv_mes = gestion_liv_mes.drop_duplicates(subset = ['usuario','credito','fecha_promesa','monto_promesa'])
gestion_liv_mes['Cliente'] = 'Liverpool'

In [19]:
#Asignamos los aranceles de liverpool
aranceles = [('DKN00REM',0.28),('DKN02REM',0.4),('DKN04REM',0.5),('SA001REM',0.15),('SA002REM',0.16)]
aranceles = pd.DataFrame(aranceles)
aranceles.columns = ['Id','Arancel']

#Agregamos aranceles al consolidado
pagos_liv2 = pd.merge(pagos_liv2,aranceles,how='left',on='Id')
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]
pagos_liv2['Fecha Pago'] = pd.to_datetime(pagos_liv2['Fecha Pago'],dayfirst=True)
pagos_liv2['Cliente'] = 'Liverpool'

In [20]:
#Obtenemos datos de los gestores de Liverpool
promesas_mes_liv = pd.pivot_table(gestion_liv_mes,index=['Cliente'],values=['monto_promesa'],aggfunc=['count',np.sum],dropna=False)
promesas_mes_liv = pd.DataFrame(promesas_mes_liv.to_records())
promesas_mes_liv.columns = ['Cliente','NumeroPromesas','SumaPromesas']
promesas_mes_liv.fillna(0,inplace=True)
promesas_mes_liv

,Cliente,NumeroPromesas,SumaPromesas
0,Liverpool,902,2988701.0


In [21]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_liv = gestion_liv_mes.loc[gestion_liv_mes['fecha_promesa']>ayer]
promesas_restantes_liv = pd.pivot_table(restantes_liv,index=['Cliente'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_liv = pd.DataFrame(promesas_restantes_liv.to_records())
promesas_restantes_liv.columns = ['Cliente','PromesasRestantes']
promesas_mes_liv.fillna(0,inplace=True)
promesas_restantes_liv

,Cliente,PromesasRestantes
0,Liverpool,474905.1


In [22]:
#Total pagos Liver registrados
pagos_totales_liv = pd.pivot_table(pagos_liv2,index=['Cliente'],values=['Monto','PagoArancel'],aggfunc=[np.sum,'count'],dropna=False)
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv.columns = ['Cliente','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_liv.fillna(0,inplace=True)
pagos_totales_liv['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv['FechaFact'] = pd.to_datetime(pagos_totales_liv['FechaFact'],dayfirst=True)
pagos_totales_liv

,Cliente,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact
0,Liverpool,716,2253072.79,783536.1734,2020-12-18


In [23]:
final_liv = pd.merge(pagos_totales_liv,promesas_mes_liv,how='left',on=['Cliente'])
final_liv = pd.merge(final_liv,promesas_restantes_liv,how='left',on=['Cliente'])
#final_liv.insert(loc=0,column='Cliente',value='Liverpool')
final_liv

,Cliente,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Liverpool,716,2253072.79,783536.1734,2020-12-18,902,2988701.0,474905.1


# Reporte Bradesco

In [24]:
#Hacemos la consulta referente a las gestiones de Bradesco
gestion_brad_mes = consulta_sql('bradescard_rpt_gestiones_detallado',mes,today)
gestion_brad_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['dictamen']=='PROMESA']
gestion_brad_mes['credito'] = pd.to_numeric(gestion_brad_mes['credito'])
gestion_brad_mes['monto_promesa'] = pd.to_numeric(gestion_brad_mes['monto_promesa'])
gestion_brad_mes = gestion_brad_mes.drop_duplicates(subset=['credito','usuario','fecha_promesa','monto_promesa'])

In [25]:
#Hacemos la union de asignacion y promesas
cols = ['monto_promesa','credito']
gestion_brad_mes[cols] = gestion_brad_mes[cols].apply(pd.to_numeric, errors='coerce')
gestion_brad_mes = gestion_brad_mes[(gestion_brad_mes['monto_promesa']>50)]
gestion_brad_mes = gestion_brad_mes[(gestion_brad_mes['monto_promesa']<500000)]
gestion_brad_mes['Cliente'] = 'Bradesco'

In [26]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [27]:
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [28]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.03)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.03)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.03)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.03)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.03)))
    
    else:
        return 0

pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)

In [29]:
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]
pagos_brad2['Cliente'] = 'Bradesco'

In [30]:
#Obtenemos datos de las promesas de Bradesco
prom_brad_mes = pd.pivot_table(gestion_brad_mes,index='Cliente',values=['monto_promesa'], aggfunc = ['count',np.sum],dropna=False)
prom_brad_mes = pd.DataFrame(prom_brad_mes.to_records())
prom_brad_mes.columns = ['Cliente','NumeroPromesas','SumaPromesas']
prom_brad_mes.fillna(0,inplace=True)
prom_brad_mes

,Cliente,NumeroPromesas,SumaPromesas
0,Bradesco,2080,3408458.65


In [31]:
gestion_brad_mes['fecha_promesa'] = gestion_brad_mes['fecha_promesa'].map(str)
gestion_brad_mes['fecha_promesa2'] = [x[:4] for x in gestion_brad_mes['fecha_promesa']]
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['fecha_promesa2']=='2020']
gestion_brad_mes['fecha_promesa'] = pd.to_datetime(gestion_brad_mes['fecha_promesa'],dayfirst=False)

In [32]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_brad = gestion_brad_mes.loc[gestion_brad_mes['fecha_promesa']>ayer]
promesas_restantes_brad = pd.pivot_table(restantes_brad,index=['Cliente'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_brad = pd.DataFrame(promesas_restantes_brad.to_records())
promesas_restantes_brad.columns = ['Cliente','PromesasRestantes']
promesas_restantes_brad.fillna(0,inplace=True)
promesas_restantes_brad

,Cliente,PromesasRestantes
0,Bradesco,561020.0


In [33]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad2,index=['Cliente'],values=['MONTO','Facturacion'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2 = pagos_totales_brad2.iloc[:,[0,1,4,3]]
pagos_totales_brad2.columns = ['Cliente','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_brad2

,Cliente,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact
0,Bradesco,1596,2601999.32,342591.8027,2020-12-17


In [34]:
final_brad = pd.merge(pagos_totales_brad2,prom_brad_mes,how='left',on=['Cliente'])
final_brad = pd.merge(final_brad,promesas_restantes_brad,how='left',on=['Cliente'])
final_brad.fillna(0,inplace=True)
final_brad

,Cliente,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Bradesco,1596,2601999.32,342591.8027,2020-12-17,2080,3408458.65,561020.0


# Reporte Banco Azteca  

In [35]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
gestion_baz = consulta_sql('baz_rpt_gestiones_detallado',mes,today)
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']

In [36]:
gestion_baz['fecha_promesa'] = gestion_baz['fecha_promesa'].map(str)
gestion_baz['fecha_promesa2'] = [x[:4] for x in gestion_baz['fecha_promesa']]
gestion_baz = gestion_baz.loc[gestion_baz['fecha_promesa2']=='2020']
gestion_baz['fecha_promesa'] = pd.to_datetime(gestion_baz['fecha_promesa'],dayfirst=False)
gestion_baz['monto_promesa'] = pd.to_numeric(gestion_baz['monto_promesa'])
gestion_baz['Cliente'] = 'Banco Azteca'

In [37]:
#Obtenemos datos de las promesas de Banco Azteca
gestion_baz = gestion_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
prom_baz_div_mes = pd.pivot_table(gestion_baz,index=['Cliente'],values=['monto_promesa'],aggfunc=('count',np.sum),dropna=False)
prom_baz_div_mes = pd.DataFrame(prom_baz_div_mes.to_records())
prom_baz_div_mes.columns = ['Cliente','NumeroPromesas','SumaPromesas']
prom_baz_div_mes.fillna(0,inplace=True)
prom_baz_div_mes

,Cliente,NumeroPromesas,SumaPromesas
0,Banco Azteca,1550,21422631.98


In [38]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_baz = gestion_baz.loc[gestion_baz['fecha_promesa']>=ayer]
promesas_restantes_baz = pd.pivot_table(restantes_baz,index=['Cliente'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_baz = pd.DataFrame(promesas_restantes_baz.to_records())
promesas_restantes_baz.columns = ['Cliente','PromesasRestantes']
promesas_restantes_baz.fillna(0,inplace=True)
promesas_restantes_baz

,Cliente,PromesasRestantes
0,Banco Azteca,10366768.49


In [39]:
pagos_baz = pd.concat([pagos_baz_26,pagos_baz_recovery],ignore_index=True)
pagos_baz = pagos_baz.loc[pagos_baz['CARGO AUTOMATICO']=='NO']
pagos_baz['Recuperado'] = pagos_baz['RECUPERACION DE CAPITAL']+pagos_baz['RECUPERACION DE MORATORIOS']
pagos_baz['Cliente'] = 'Banco Azteca'

In [40]:
pagos_totales_baz = pd.pivot_table(fact_baz,index=['Cliente'],values=['Total Recuperado','Total Comisión'],aggfunc=[np.sum],dropna=False)
pagos_totales_baz = pd.DataFrame(pagos_totales_baz.to_records())
pagos_totales_baz.columns = ['Cliente','Recuperado','SumaPagos']
porcentaje_prom_arancel_baz = pagos_totales_baz['Recuperado']/pagos_totales_baz['SumaPagos']
porcentaje_prom_arancel_baz

0    0.169313
dtype: float64

In [41]:
pagos_fact_baz = pd.pivot_table(pagos_baz,index=['Cliente'],values=['Recuperado'],aggfunc=['count',np.sum])
pagos_fact_baz = to_dataframe(pagos_fact_baz)
pagos_fact_baz.columns = ['Cliente','NumeroPagos','SumaPagos']
pagos_fact_baz['SumaFacturacion'] = pagos_fact_baz['SumaPagos']*porcentaje_prom_arancel_baz
pagos_fact_baz['FechaFact'] = max(pagos_baz['DIA'])
pagos_fact_baz

,Cliente,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact
0,Banco Azteca,1748,1648177,279057.955072,2020-12-17


In [42]:
final_baz = pd.merge(pagos_fact_baz,prom_baz_div_mes,how='left',on=['Cliente'])
final_baz = pd.merge(final_baz,promesas_restantes_baz,how='left',on=['Cliente'])
final_baz.fillna(0,inplace=True)
final_baz

,Cliente,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Banco Azteca,1748,1648177,279057.955072,2020-12-17,1550,21422631.98,10366768.49


# Reporte Banco Azteca Renta

In [43]:
pagos_renta = pd.pivot_table(pagos_baz_renta2,index=['Cliente'],values=['FNMONTOPAGADO'],aggfunc=['count',np.sum],dropna=False)
pagos_renta = pd.DataFrame(pagos_renta.to_records())
pagos_renta.columns = ['Cliente','NumeroPagos','SumaPagos']
pagos_renta['FechaFact'] = max(pagos_baz_renta2['FDFECHAABONO'])
pagos_renta

,Cliente,NumeroPagos,SumaPagos,FechaFact
0,Renta Azteca,38625,26085305.45,2020-12-18


# Reporte Credifiel

In [44]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred = consulta_sql('credifiel_rpt_gestiones_detallado',mes,today)
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred['credito'] = [x[3:] for x in gestion_cred['credito']]
gestion_cred['credito'] = pd.to_numeric(gestion_cred['credito'])
gestion_cred['Cliente'] = 'Credifiel'
#gestion_cred.head(3)

In [45]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_pagos_cred = pd.merge(pagos_cred2,asignacion_cred,how='left',left_on=['ID'],right_on=['CREDITO'])
consolidado_pagos_cred.drop('CREDITO',1,inplace=True)
consolidado_pagos_cred['ARANCEL'] = np.where(consolidado_pagos_cred['ARANCEL'].isna(),np.mean(consolidado_pagos_cred['ARANCEL']),consolidado_pagos_cred['ARANCEL'])
consolidado_pagos_cred['FACTURACION'] = consolidado_pagos_cred['IMPORTE']*consolidado_pagos_cred['ARANCEL']
consolidado_pagos_cred['CLIENTE'] = 'Credifiel'
consolidado_pagos_cred.head(1)

,ID,FECHA,IMPORTE,ARANCEL,FACTURACION,CLIENTE
0,74246,2020-12-01,388.51,0.191569,74.426327,Credifiel


In [46]:
#Obtenemos datos de las promesas de Bradesco
prom_cred_div_mes = pd.pivot_table(gestion_cred,index=['Cliente'],values=['monto_promesa'], aggfunc=['count',np.sum],dropna=False)
prom_cred_div_mes = pd.DataFrame(prom_cred_div_mes.to_records())
prom_cred_div_mes.columns = ['Cliente','NumeroPromesas','SumaPromesas']
#prom_brad_div_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = prom_brad_gestor_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
prom_cred_div_mes

,Cliente,NumeroPromesas,SumaPromesas
0,Credifiel,107,342106.3


In [47]:
#Obtenemos las promesas restantes a partir del día de hoy
gestion_cred['fecha_promesa'] = pd.to_datetime(gestion_cred['fecha_promesa'],dayfirst=False)
restantes_cred = gestion_cred.loc[gestion_cred['fecha_promesa']>=ayer]
promesas_restantes_cred = pd.pivot_table(restantes_cred,index=['Cliente'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_cred = pd.DataFrame(promesas_restantes_cred.to_records())
promesas_restantes_cred.columns = ['Cliente','PromesasRestantes']
promesas_restantes_cred.fillna(0,inplace=True)
#promesas_restantes_brad[['SumaPromesas']] = promesas_restantes_brad[['SumaPromesas']].applymap("{0:.2f}".format)
promesas_restantes_cred

,Cliente,PromesasRestantes
0,Credifiel,125984.69


In [48]:
pagos_totales_cred = pd.pivot_table(consolidado_pagos_cred,index=['CLIENTE'],values=['IMPORTE','FACTURACION'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
pagos_totales_cred = pagos_totales_cred.iloc[:,[0,1,4,3]]
pagos_totales_cred.columns = ['Cliente','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_cred.fillna(0,inplace=True)
#pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
pagos_totales_cred['FechaFact'] = max(consolidado_pagos_cred['FECHA'])
pagos_totales_cred

,Cliente,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact
0,Credifiel,65,180437.74,36731.721398,2020-12-15


In [49]:
final_cred = pd.merge(pagos_totales_cred,prom_cred_div_mes,how='left',on=['Cliente'])
final_cred = pd.merge(final_cred,promesas_restantes_cred,how='left',on=['Cliente'])
final_cred.fillna(0,inplace=True)
final_cred

,Cliente,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Credifiel,65,180437.74,36731.721398,2020-12-15,107,342106.3,125984.69


# Reporte final Naucalpan

In [59]:
final_naucalpan = pd.concat([final_liv,final_brad],ignore_index=True)
final_naucalpan = pd.concat([final_naucalpan,final_cred],ignore_index=True)
final_naucalpan = pd.concat([final_naucalpan,final_baz],ignore_index=True)
final_naucalpan = pd.merge(final_naucalpan,meta,how='left',on='Cliente')
final_naucalpan['Avance%'] = final_naucalpan['SumaFacturacion']/final_naucalpan['Meta']
final_naucalpan.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan.insert(loc=0,column='Pais',value='Mexico')
final_naucalpan['TendenciaCierre1'] = [(30/x.day)*y for x,y in zip(final_naucalpan['FechaFact'],final_naucalpan['SumaFacturacion'])]
cols = ['SumaPagos','SumaFacturacion','SumaPromesas','PromesasRestantes','TendenciaCierre1']
final_naucalpan[cols] = final_naucalpan[cols].round(2)
final_naucalpan.fillna(0,inplace=True)
final_naucalpan = final_naucalpan[[]]
final_naucalpan

,Pais,Sucursal,Cliente,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes,Meta,Avance%,TendenciaCierre1
0,Mexico,Naucalpan,Liverpool,716,2253072.79,783536.17,2020-12-18,902,2988701.00,474905.10,1000000.0,0.783536,1305893.62
1,Mexico,Naucalpan,Bradesco,1596,2601999.32,342591.80,2020-12-17,2080,3408458.65,561020.00,1200000.0,0.285493,604573.77
2,Mexico,Naucalpan,Credifiel,65,180437.74,36731.72,2020-12-15,107,342106.30,125984.69,120000.0,0.306098,73463.44
3,Mexico,Naucalpan,Banco Azteca,1748,1648177.00,279057.96,2020-12-17,1550,21422631.98,10366768.49,0.0,0.000000,492455.21


In [51]:
#final_naucalpan.to_csv(''+ruta_guardar+'/Reporte Sucursal/Reporte sucursal '+today2+'.csv',index=False,header=True)